<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/result/(%ED%99%9C%EC%9A%A9)_%EB%AA%A8%EB%8D%B8%EB%B6%88%EB%9F%AC%EC%99%80%EC%84%9C_%EC%A0%9C%EC%B6%9C%EC%9A%A9%ED%8C%8C%EC%9D%BC_%EC%9E%91%EC%84%B1_C2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

     |████████████████████████████████| 47.3 MB 1.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 8.0 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595725 sha256=b68577e207084f4bdc55c774e30ca2b71815dc44b8c2063eae813a39809858d4
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 9.4 MB/s 
     |████████████████████████████████| 769 kB 7.2 MB/s 
     |████████████████████████████████| 3.0 MB 41.3 MB/s 
     |████████████████████████████████| 895 kB 49.9 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-g9ot1mob
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pi

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화		 df_E_inc.csv	   F_inc_model_2.pt   P_inc_model.pt
'Colab Notebooks'	'# df_G_inc.csv'   G_inc_model.pt     Q_inc_model.pt
 C_ori_model.pt		 df_G_inc.csv	   I_ori_model.pt     S_ori_model.pt
'답안 작성용 파일.csv'	 df_J_inc.csv	   J_inc_model.pt     데이터.zip
 df_A_inc.csv		 df_K_inc.csv	  'My Drive'
 df_B_inc.csv		 df_N_inc.csv	   N_inc_model.pt


In [4]:
!unzip /content/gdrive/'My Drive'/산업분류자동화/'모델 개발용 자료'.zip -d /content/dataset
!unzip /content/gdrive/'My Drive'/산업분류자동화/'실습용 자료'.zip -d /content/dataset

Archive:  /content/gdrive/My Drive/산업분류자동화/모델 개발용 자료.zip
  inflating: /content/dataset/2. ╕Ё╡и┐ы░│╣▀┐ы└┌╖с.csv  
Archive:  /content/gdrive/My Drive/산업분류자동화/실습용 자료.zip
1. 실습용자료.txt:  mismatching "local" filename (1. ьЛдьК╡ьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/1. 실습용자료.txt  


In [5]:
import pandas as pd
ts=pd.read_csv('/content/gdrive/MyDrive/답안 작성용 파일.csv' ,encoding = "euc-kr")

In [6]:
test=ts[ts['digit_1']=='C']
test=test.reset_index(drop=True)

# 제출용 파일 만들기

In [7]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=85,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5 #추후 num_epochs 증가할 필요 있음
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [8]:
model = torch.load('/content/gdrive/MyDrive/C_ori_model.pt')

model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [9]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)  
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
        return np.argmax(logits)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


# 제출하기

In [10]:
test['text']=test['text_obj'].map(str)+" "+test['text_mthd'].map(str)+" "+test['text_deal'].map(str)

i=[]
for sentence in test['text']:
    a=sentence.replace('nan', '')
    i.append(a)
test['text']=pd.DataFrame(i)

test=test.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제

test.head(3)

,AI_id,digit_1,digit_2,digit_3,text
0,id_000006,C,NaN,NaN,"철,아크릴,포맥스 스크린인쇄 명판"
1,id_000008,C,NaN,NaN,스테인레스를 프레스가공하여제조 주방용품
2,id_000010,C,NaN,NaN,"약품(화공), 미싱 완성품입고, 수선"


In [11]:
result=[]
for sentence in test['text']:
  temp=predict(sentence)
  result.append(temp)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [13]:
a=pd.DataFrame(result, columns=['digit_3'])
a.loc[(a['digit_3']==0), 'digit_3']=101
a.loc[(a['digit_3']==1), 'digit_3']=102
a.loc[(a['digit_3']==2), 'digit_3']=103
a.loc[(a['digit_3']==3), 'digit_3']=104
a.loc[(a['digit_3']==4), 'digit_3']=105
a.loc[(a['digit_3']==5), 'digit_3']=106
a.loc[(a['digit_3']==6), 'digit_3']=107
a.loc[(a['digit_3']==7), 'digit_3']=108
a.loc[(a['digit_3']==8), 'digit_3']=111
a.loc[(a['digit_3']==9), 'digit_3']=112
a.loc[(a['digit_3']==10), 'digit_3']=120
a.loc[(a['digit_3']==11), 'digit_3']=131
a.loc[(a['digit_3']==12), 'digit_3']=132
a.loc[(a['digit_3']==13), 'digit_3']=133
a.loc[(a['digit_3']==14), 'digit_3']=134
a.loc[(a['digit_3']==15), 'digit_3']=139
a.loc[(a['digit_3']==16), 'digit_3']=141
a.loc[(a['digit_3']==17), 'digit_3']=142
a.loc[(a['digit_3']==18), 'digit_3']=143
a.loc[(a['digit_3']==19), 'digit_3']=144
a.loc[(a['digit_3']==20), 'digit_3']=151
a.loc[(a['digit_3']==21), 'digit_3']=152
a.loc[(a['digit_3']==22), 'digit_3']=161
a.loc[(a['digit_3']==23), 'digit_3']=162
a.loc[(a['digit_3']==24), 'digit_3']=163
a.loc[(a['digit_3']==25), 'digit_3']=171
a.loc[(a['digit_3']==26), 'digit_3']=172
a.loc[(a['digit_3']==27), 'digit_3']=179
a.loc[(a['digit_3']==28), 'digit_3']=181
a.loc[(a['digit_3']==29), 'digit_3']=182
a.loc[(a['digit_3']==30), 'digit_3']=191
a.loc[(a['digit_3']==31), 'digit_3']=192
a.loc[(a['digit_3']==32), 'digit_3']=201
a.loc[(a['digit_3']==33), 'digit_3']=202
a.loc[(a['digit_3']==34), 'digit_3']=203
a.loc[(a['digit_3']==35), 'digit_3']=204
a.loc[(a['digit_3']==36), 'digit_3']=205
a.loc[(a['digit_3']==37), 'digit_3']=211
a.loc[(a['digit_3']==38), 'digit_3']=212
a.loc[(a['digit_3']==39), 'digit_3']=213
a.loc[(a['digit_3']==40), 'digit_3']=221
a.loc[(a['digit_3']==41), 'digit_3']=222
a.loc[(a['digit_3']==42), 'digit_3']=231
a.loc[(a['digit_3']==43), 'digit_3']=232
a.loc[(a['digit_3']==44), 'digit_3']=233
a.loc[(a['digit_3']==45), 'digit_3']=239
a.loc[(a['digit_3']==46), 'digit_3']=241
a.loc[(a['digit_3']==47), 'digit_3']=242
a.loc[(a['digit_3']==48), 'digit_3']=243
a.loc[(a['digit_3']==49), 'digit_3']=251
a.loc[(a['digit_3']==50), 'digit_3']=252
a.loc[(a['digit_3']==51), 'digit_3']=259
a.loc[(a['digit_3']==52), 'digit_3']=261
a.loc[(a['digit_3']==53), 'digit_3']=262
a.loc[(a['digit_3']==54), 'digit_3']=263
a.loc[(a['digit_3']==55), 'digit_3']=264
a.loc[(a['digit_3']==56), 'digit_3']=265
a.loc[(a['digit_3']==57), 'digit_3']=266
a.loc[(a['digit_3']==58), 'digit_3']=271
a.loc[(a['digit_3']==59), 'digit_3']=272
a.loc[(a['digit_3']==60), 'digit_3']=273
a.loc[(a['digit_3']==61), 'digit_3']=274
a.loc[(a['digit_3']==62), 'digit_3']=281
a.loc[(a['digit_3']==63), 'digit_3']=282
a.loc[(a['digit_3']==64), 'digit_3']=283
a.loc[(a['digit_3']==65), 'digit_3']=284
a.loc[(a['digit_3']==66), 'digit_3']=285
a.loc[(a['digit_3']==67), 'digit_3']=289
a.loc[(a['digit_3']==68), 'digit_3']=291
a.loc[(a['digit_3']==69), 'digit_3']=292
a.loc[(a['digit_3']==70), 'digit_3']=301
a.loc[(a['digit_3']==71), 'digit_3']=302
a.loc[(a['digit_3']==72), 'digit_3']=303
a.loc[(a['digit_3']==73), 'digit_3']=304
a.loc[(a['digit_3']==74), 'digit_3']=311
a.loc[(a['digit_3']==75), 'digit_3']=312
a.loc[(a['digit_3']==76), 'digit_3']=313
a.loc[(a['digit_3']==77), 'digit_3']=319
a.loc[(a['digit_3']==78), 'digit_3']=320
a.loc[(a['digit_3']==79), 'digit_3']=331
a.loc[(a['digit_3']==80), 'digit_3']=332
a.loc[(a['digit_3']==81), 'digit_3']=333
a.loc[(a['digit_3']==82), 'digit_3']=334
a.loc[(a['digit_3']==83), 'digit_3']=339
a.loc[(a['digit_3']==84), 'digit_3']=340


In [14]:
test['digit_3']=a

In [15]:
test.to_csv('result_C.csv', index=False, encoding='utf-8-sig')